In [1]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


Bism  
# Imports

In [2]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook


#SKLEARN
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as sklm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE

#KERAS
from keras import models
from keras.models import Model, load_model
from keras import callbacks
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


# Implementation 10  (PHONE LEVEL)
## Using Cambridge as Training set
## L2 as val+test set
## AUTOENCODER PER PHONE

In [0]:
wd = './drive/My Drive/Thesis Work/LIBRISPEECH'
phones = np.load(wd+'/phoneList.npy')
phones = phones[(phones != 'sil') & (phones != 'sp') & (phones != 'spn')]
len(phones)

39

In [0]:
phone = "R"

### Training

#### X_TRAIN

In [0]:
X_train = np.concatenate((np.load(wd+'/Libri Phones Train/'+phone+'_X1.npy'),np.load(wd+'/Libri Phones Train/'+phone+'_X2.npy')))
np.random.shuffle(X_train) #very important
print("Shape of X_train: ",X_train.shape)
X_train = X_train[:80000]
print("Shape of X_train after reduction: ",X_train.shape)

Shape of X_train:  (146203, 29, 13, 3)
Shape of X_train after reduction:  (80000, 29, 13, 3)


#### MODEL

In [0]:
#commented to avoid accedental runs
#Making Min Max value files for normalization for every phone
'''import gc
wd = './drive/My Drive/Thesis Work/LIBRISPEECH'
phones = np.load(wd+'/phoneList.npy')
for phone in tqdm_notebook(phones):
  if phone != 'AH':
    X_train = np.concatenate((np.load(wd+'/Libri Phones Train/'+phone+'_X1.npy'),
                              np.load(wd+'/Libri Phones Train/'+phone+'_X2.npy')))
    Xmax = np.max(X_train,(0,1,2))
    Xmin = np.min(X_train,(0,1,2))
    del X_train
    gc.collect()
    np.save(wd+"/phone_MinMax/min_"+phone+".npy", Xmin)
    np.save(wd+"/phone_MinMax/max_"+phone+".npy", Xmax)
    
#AH too big only 1 batch taken
X_train = np.load(wd+'/Libri Phones Train/'+'AH'+'_X1.npy')
Xmax = np.max(X_train,(0,1,2))
Xmin = np.min(X_train,(0,1,2))
del X_train
gc.collect()
np.save(wd+"/phone_MinMax/min_"+'AH'+".npy", Xmin)
np.save(wd+"/phone_MinMax/max_"+'AH'+".npy", Xmax)''';

In [0]:
import gc
gc.collect()

9

In [0]:
Xmax = np.load(wd+"/phone_MinMax/max_"+phone+".npy")
Xmin = np.load(wd+"/phone_MinMax/min_"+phone+".npy")
X_train = (X_train - Xmin) / (Xmax - Xmin)
print(phone)
print(X_train.shape)

R
(80000, 29, 13, 3)


In [0]:
print(np.max(X_train,axis=(0,1,2)))
print(np.min(X_train,axis=(0,1,2))) #mean is not of 80000

[1.0 0.9125881955052886 0.9819936029848674]
[0.004266285487972951 0.038867298854895385 0.0]


In [0]:
np.nan in X_train

False

In [0]:
'''noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_train_noisy = np.clip(x_train_noisy, -1, 1.)''';

In [0]:
input_img = Input(shape=(29, 13, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(64, (2, 2), activation='relu', padding='valid')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (7, 3, 128)

x = BatchNormalization()(encoded)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = BatchNormalization()(x)
decoded = Conv2DTranspose(3, (2, 2), activation='relu', padding='valid')(x)

autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer= keras.optimizers.Adam(), loss='mean_squared_error');

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 29, 13, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 12, 64)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 6, 64)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 6, 64)         256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 6, 128)        73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 3, 128)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 3, 128)         512       
__________

In [0]:
callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)]
autoencoder.fit(X_train, X_train, batch_size=32, epochs=50, callbacks=callbacklist)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
80000/80000 [==============================] - 91s 1ms/step - loss: 0.0086
Epoch 2/50
80000/80000 [==============================] - 90s 1ms/step - loss: 6.3750e-04
Epoch 3/50
80000/80000 [==============================] - 85s 1ms/step - loss: 3.6681e-04
Epoch 4/50
80000/80000 [==============================] - 90s 1ms/step - loss: 2.6868e-04
Epoch 5/50
80000/80000 [==============================] - 90s 1ms/step - loss: 2.3023e-04

Epoch 00005: loss improved from inf to 0.00023, saving model to ./jawadmodel.h5
Epoch 6/50
80000/80000 [==============================] - 81s 1ms/step - loss: 2.0757e-04
Epoch 7/50
80000/80000 [==============================] - 82s 1ms/step - loss: 1.9297e-04
Epoch 8/50
80000/80000 [==============================] - 82s 1ms/step - loss: 1.8522e-04
Epoch 9/50
80000/80000 [==============================] - 82s 1ms/step - loss: 1.7851e-04
Epoch 10/50
80000/80000 [==============================] - 82s 1m

KeyboardInterrupt: ignored

In [0]:
modelLoss = _.history['loss'][-1]
modelLoss

AttributeError: ignored

In [0]:
phone

'R'

In [0]:
wd = "'./drive/My Drive/Thesis Work/LIBRISPEECH'"

In [0]:
!cp ./jawadmodel.h5 $wd"/autoEncoder Models/"$phone".h5"

### Validation

In [0]:
wd = "'./drive/My Drive/Thesis Work/LIBRISPEECH'"

In [0]:
X_val= np.load(wd[1:-1]+'/Libri Phones Dev/'+phone+'_X.npy')
np.random.shuffle(X_val)
y_val = np.ones(X_val.shape[0])
print("Total Samples validation (only correct phone): ", X_val.shape[0])
print("Shape of y_val (only correct yet): ", y_val.shape)

Total Samples validation (only correct phone):  8098
Shape of y_val (only correct yet):  (8098,)


In [0]:
for p in tqdm_notebook(phones):
  if p != phone:
    notPhone = np.load(wd[1:-1]+'/Libri Phones Dev/'+p+'_X.npy')
    np.random.shuffle(notPhone)
    notPhone = notPhone[:int(np.ceil(y_val.shape[0]/38))]
    X_val = np.vstack((X_val,notPhone))
    y_val = np.hstack((y_val,np.zeros(notPhone.shape[0])))
    
print("Shape of y_val after adding mispronunciations: ",y_val.shape)
print("Shape of X_val after adding mispronunciations: ",X_val.shape)
print("Shape of X_val: ",X_val.shape)


Shape of y_val after adding mispronunciations:  (20955,)
Shape of X_val after adding mispronunciations:  (20955, 29, 13, 3)
Shape of X_val:  (20955, 29, 13, 3)


In [0]:
!ls $wd"/autoEncoder Models/"

AA.h5  R.h5


In [0]:
#MIN-MAX normalization
Xmax = np.load(wd[1:-1]+"/phone_MinMax/max_"+phone+".npy")
Xmin = np.load(wd[1:-1]+"/phone_MinMax/min_"+phone+".npy")
X_val = (X_val - Xmin) / (Xmax - Xmin)
print(phone)
print(X_val.shape)
print(y_val.shape)

R
(20955, 29, 13, 3)
(20955,)


In [0]:
'''noise_factor = 0.5
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 
x_test_noisy = np.clip(x_test_noisy, -1, 1)''';

In [0]:
autoencoder = load_model(wd[1:-1]+"/autoEncoder Models/"+phone+'.h5') #load previous one without noise

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
autoencoder.evaluate(X_val[y_val == 1],X_val[y_val == 1],batch_size=1)

8098/8098 [==============================] - 21s 3ms/step


0.0001527248129135007

In [0]:
autoencoder.evaluate(X_val[y_val == 0],X_val[y_val == 0],batch_size=1)

12857/12857 [==============================] - 30s 2ms/step


0.00021009655146417866

In [0]:
validationSetLength = X_val.shape[0]
accs = []
f1mis = []

for threshold in np.arange(0.0001,0.0003,0.00001):
  y_pred = [1 if autoencoder.evaluate(X_val[i:i+1],X_val[i:i+1],verbose=0)<=threshold\
   else 0 for i in range(validationSetLength)]
  
  accs.append(sklm.accuracy_score(y_val,y_pred))
  f1mis.append(sklm.precision_recall_fscore_support(y_val,y_pred)[2][0])

print("Max Achievable Accuracy (Overall): ", max(accs))
print("Max Achievable Accuracy (Overall) on Threshold: ", np.argmax(accs))

print("Max Achievable F-1 score Threshold: ", np.argmax(f1mis))
print("Max Achievable Accuracy On This Threshold: ", accs[np.argmax(f1mis)])
print("Max Achievable F-1 score for Mispronunciations: ", max(f1mis))

Max Achievable Accuracy (Overall):  0.6345979479837748
Max Achievable Accuracy (Overall) on Threshold:  3
Max Achievable F-1 score Threshold:  0
Max Achievable Accuracy On This Threshold:  0.6215223097112861
Max Achievable F-1 score for Mispronunciations:  0.7491698029665707


### Testing

In [0]:
wd = "'./drive/My Drive/Thesis Work/LIBRISPEECH'"
autoencoder = load_model(wd[1:-1]+"/autoEncoder Models/"+phone+'.h5') #load previous one without noise

In [0]:
X_test = np.load(wd[1:-1]+'/Libri Phones Test/'+phone+'_X.npy')
np.random.shuffle(X_test)
y_test = np.ones(X_test.shape[0])
print("Total Samples test (only correct phone): ", X_test.shape[0])
print("Shape of y_test (only correct yet): ", y_test.shape)

Total Samples test (only correct phone):  8058
Shape of y_test (only correct yet):  (8058,)


In [0]:
for p in tqdm_notebook(phones):
  if p != phone:
    notPhone = np.load(wd[1:-1]+'/Libri Phones Test/'+p+'_X.npy')
    np.random.shuffle(notPhone)
    notPhone = notPhone[:int(np.ceil(y_test.shape[0]/38))]
    X_test = np.vstack((X_test,notPhone))
    y_test = np.hstack((y_test,np.zeros(notPhone.shape[0])))

print("Shape of y_test after adding mispronunciations: ",y_test.shape)
print("Shape of X_test after adding mispronunciations: ",X_test.shape)


Shape of y_test after adding mispronunciations:  (20817,)
Shape of X_test after adding mispronunciations:  (20817, 29, 13, 3)


In [0]:
#MIN-MAX normalization
Xmax = np.load(wd[1:-1]+"/phone_MinMax/max_"+phone+".npy")
Xmin = np.load(wd[1:-1]+"/phone_MinMax/min_"+phone+".npy")
X_test = (X_test - Xmin) / (Xmax - Xmin)
print(X_test.shape)
print(y_test.shape)

(20817, 29, 13, 3)
(20817,)


In [0]:
threshold = 0.0001

y_pred = [1 if autoencoder.evaluate(X_test[i:i+1],X_test[i:i+1],verbose=0)<=threshold\
 else 0 for i in range(X_test.shape[0])]

In [0]:
print("Phone: ",phone)
print(classification_report(y_test, y_pred, target_names=['Mispronounced','Correct']))
print(confusion_matrix(y_test, y_pred, labels=range(2)))

print(sklm.roc_curve(y_test,y_pred))
print('FAR = ', sklm.roc_curve(y_test,y_pred)[0][1]*100,'%')
print('FRR = ', (1 - sklm.roc_curve(y_test,y_pred)[1][1])*100,'%')
print("AUC_SCORE = ", sklm.roc_auc_score(y_test,y_pred)*100,'%')

Phone:  R
               precision    recall  f1-score   support

Mispronounced       0.64      0.93      0.76     12759
      Correct       0.59      0.15      0.24      8058

    micro avg       0.63      0.63      0.63     20817
    macro avg       0.61      0.54      0.50     20817
 weighted avg       0.62      0.63      0.56     20817

[[11912   847]
 [ 6824  1234]]
(array([0.        , 0.06638451, 1.        ]), array([0.        , 0.15313974, 1.        ]), array([2, 1, 0]))
FAR =  6.638451289285994 %
FRR =  84.68602630925788 %
AUC_SCORE =  54.33776120072806 %


*CANNOT COMPARE RESULTS:*  
The Interspeech one has a **different dataset** and shows results on phone level (as every phone is a separate class there)

The l2-arctic one shows no results

I am the first one to show results on this dataset